In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ls -lha ../data/*.hdf

-rw-rw-r-- 1 ubuntu ubuntu 7.7M Apr 19 17:21 ../data/click_count_by_ip.hdf
-rw-rw-r-- 1 ubuntu ubuntu 158M Apr 19 17:29 ../data/click_count_by_ip_os_device_dow.hdf
-rw-rw-r-- 1 ubuntu ubuntu 529M Apr 19 17:33 ../data/click_count_by_ip_os_device_dow_hour.hdf
-rw-rw-r-- 1 ubuntu ubuntu  18G Apr 19 10:09 ../data/click_data.hdf
-rw-rw-r-- 1 ubuntu ubuntu 3.1G Apr 19 20:16 ../data/click_time_interval_by_ip.hdf
-rw-rw-r-- 1 ubuntu ubuntu 3.1G Apr 19 20:48 ../data/click_time_interval_by_ip_os_device.hdf
-rw-rw-r-- 1 ubuntu ubuntu 3.1G Apr 19 20:58 ../data/click_time_interval_by_ip_os_device_dow.hdf
-rw-rw-r-- 1 ubuntu ubuntu 3.1G Apr 19 21:05 ../data/click_time_interval_by_ip_os_device_dow_hour.hdf
-rw-rw-r-- 1 ubuntu ubuntu 7.6G Apr 19 11:00 ../data/click_times.hdf
-rw-rw-r-- 1 ubuntu ubuntu 3.0G Apr 19 11:26 ../data/ip_os_device.hdf
-rw-rw-r-- 1 ubuntu ubuntu 4.6G Apr 19 17:02 ../data/ip_os_device_dow.hdf
-rw-rw-r-- 1 ubuntu ubuntu 6.1G Apr 19 18:37 ../data/ranks.hdf


In [3]:
def read_hdf(name:str):
    return pd.read_hdf("../data/{}.hdf".format(name), name)

In [4]:
%%time
ids = pd.concat([
    read_hdf("ip_os_device"),
    read_hdf("ip_os_device_dow")
], axis=1)

CPU times: user 19.8 s, sys: 10 s, total: 29.8 s
Wall time: 2min 24s


In [5]:
ids

,ip_os_device,ip_os_device_dow,ip_os_device_dow_hour
0,832300130001,8323001300010,832300130001014
1,173570190001,1735701900010,173570190001014
2,358100130001,3581001300010,358100130001014
3,457450130001,4574501300010,457450130001014
4,1610070130001,16100701300010,1610070130001014
5,187870160001,1878701600010,187870160001014
6,1030220230001,10302202300010,1030220230001014
7,1142210190001,11422101900010,1142210190001014
8,1659700130001,16597001300010,1659700130001014
9,745440220001,7454402200010,745440220001014


In [6]:
%%time
click_count_by_ip=read_hdf("click_count_by_ip")
click_count_by_ip_os_device_dow=read_hdf("click_count_by_ip_os_device_dow")
click_count_by_ip_os_device_dow_hour=read_hdf("click_count_by_ip_os_device_dow_hour")

CPU times: user 196 ms, sys: 252 ms, total: 448 ms
Wall time: 10.2 s


In [11]:
%%time
ids = pd.merge(ids, click_count_by_ip_os_device_dow, on='ip_os_device_dow', how='left', sort=False)

CPU times: user 1min 3s, sys: 11.2 s, total: 1min 14s
Wall time: 1min 14s


In [13]:
%%time
ids = pd.merge(ids, click_count_by_ip_os_device_dow_hour, on='ip_os_device_dow_hour', how='left', sort=False)

CPU times: user 47.3 s, sys: 12.9 s, total: 1min
Wall time: 1min


In [14]:
%%time
ip = read_hdf("click_data")
ip = ip[["ip"]]

CPU times: user 14.4 s, sys: 9.57 s, total: 23.9 s
Wall time: 2min 53s


In [15]:
%%time
ids = pd.concat([ids, ip], axis=1)

CPU times: user 5.78 s, sys: 3.96 s, total: 9.74 s
Wall time: 9.73 s


In [16]:
%%time
ids = pd.merge(ids, click_count_by_ip, on='ip', how='left', sort=False)

CPU times: user 1min 1s, sys: 12.8 s, total: 1min 14s
Wall time: 1min 14s


In [18]:
ids[["click_count_by_ip_os_device_dow", "click_count_by_ip_os_device_dow_hour", "click_count_by_ip"]].to_hdf("../data/click_counts.hdf", "click_counts")